In [ ]:
import os

import astropy.units as u
import matplotlib.pyplot as plt
import numpy as np
from scipy.special import roots_legendre
from scipy.integrate import quad_vec
import pyccl as ccl

from SSLimPy.interface import sslimpy
from SSLimPy.cosmology import cosmology
from SSLimPy.cosmology import halo_model
from SSLimPy.LIMsurvey import higher_order
from SSLimPy.utils.utils import linear_interpolate, addVectors

import SSLimPy.LIMsurvey.power_spectrum as sps
import SSLimPy.cosmology.astro as sastro
import SSLimPy.LIMsurvey.covariance as scov
import SSLimPy.interface.survey_specs as sss

In [ ]:
import seaborn
seaborn.set_theme(rc={'axes.edgecolor': 'black', 'xtick.color': 'black', 'ytick.color': 'black',})
C = seaborn.color_palette("Paired")
C

In [ ]:
envkey = "OMP_NUM_THREADS"
# Set this environment variable to the number of available cores in your machine,
# to get a fast execution of the Einstein Boltzmann Solver
print("The value of {:s} is: ".format(envkey), os.environ.get(envkey))
os.environ[envkey] = str(12)
print("The value of {:s} is: ".format(envkey), os.environ.get(envkey))

In [ ]:
cosmo_dict = {
    "h": 0.7,
    "ns": 0.96,
    "sigma8": 0.82,
    "Omegab": 0.05,
    "Omegam": 0.32,
    "mnu": 0.06,
    "Neff": 3.044
}

halo_dict = {
    "hmf_model":"ST",
    "bias_model": "b1",
    "nR" : 256,
    "Rmin": 1e-3 * u.Mpc,
    "Rmax": 1e3 * u.Mpc,
    "bloating": "Mead20",
}

In [ ]:
settings = {"code":"class",
            "do_RSD" : True,
            "nonlinearRSD" : True,
            "FoG_damp" : "ISTF_like",
            "halo_model_PS" : True,
            "Smooth_window" : False,
            "nk":200,
            "kmax": 50*u.Mpc**-1,
            "kmin": 1e-5*u.Mpc**-1,
            "Smooth_resolution": False,
            }

sslimpy.sslimpy(settings_dict=settings)

In [ ]:
s_cosmo = cosmology.CosmoFunctions(cosmopars=cosmo_dict)
s_halo = halo_model.HaloModel(s_cosmo, halo_dict)

k = s_halo.k
print(np.min(k), np.max(k))

In [ ]:
pobs_settings = {"nk":100,
            "kmax": 50*u.Mpc**-1,
            "kmin": 1e-4*u.Mpc**-1,
            "nmu":20
            }

In [ ]:
survey_specs = sss.SurveySpecifications(dict(), s_cosmo)
astro = sastro.AstroFunctions(s_halo, survey_specs)
pobs = sps.PowerSpectra(astro, settings=pobs_settings)
cov = scov.nonGuassianCov(pobs)

In [ ]:
k = pobs.k
Pk = s_cosmo.matpow(k, 0)

In [ ]:
args = (0.0010577432818586437, 0.0010577432818586437, -0.9681602395076261, 0.9681602395076261,1, -0.2, 0.2, -1.2, 0.4, 2, 0.4, 0.7, 0, 0,)
higher_order.integrand_4h(3.013364382014699, *args, k.value, Pk.value)

In [ ]:
mu1 = np.linspace(-1, 1)
phis = np.linspace(-np.pi, np.pi)

phiintegrnd1 = np.empty_like(phis)
phiintegrnd2 = np.empty_like(phis)
I1 = np.zeros_like(mu1)
I2 = np.zeros_like(mu1)

for im, mi in enumerate(mu1):
    args = (0.4885185780734908, 0.7451451590319873, 0.7, mi, 1, -0.2, 0.2, -1.2, 0.4, 2, 0.4, 0.7, 0, 0,)
    args2 = (0.4885185780734908, 0.7451451590319873, -0.7, mi, 1, -0.2, 0.2, -1.2, 0.4, 2, 0.4, 0.7, 0, 0,)
    
    for ip, pi in enumerate(phis):
        phiintegrnd1[ip] = higher_order.integrand_4h(pi, *args, k.value, Pk.value)
        phiintegrnd2[ip] = higher_order.integrand_4h(pi, *args2, k.value, Pk.value)
    I1[im] = np.trapz(phiintegrnd1, phis)/(2 * np.pi)
    I2[im] = np.trapz(phiintegrnd2, phis)/(2 * np.pi)


plt.plot(mu1, I1)
plt.plot(mu1, I2)

# Further tests

In [ ]:
Omega_c = s_cosmo.Omega(0, "clustering") - 0.05
cosmo = ccl.Cosmology(Omega_c=Omega_c, Omega_b=0.05, h=0.7, sigma8=0.82, n_s=0.96, m_nu=0.06, Neff=3.044)

cM = ccl.halos.ConcentrationDiemer15()
nM = ccl.halos.MassFuncSheth99(mass_def=ccl.halos.MassDef200c, mass_def_strict=False)
bM = ccl.halos.HaloBiasSheth99(mass_def=ccl.halos.MassDef200c, mass_def_strict=False)
pM = ccl.halos.HaloProfileNFW(mass_def=ccl.halos.MassDef200c, concentration=cM, fourier_analytic=True)

hmc = ccl.halos.HMCalculator(mass_function=nM, halo_bias=bM)

Linear power spectra

In [ ]:
linpk_SL = s_cosmo.matpow(k, 0, nonlinear=False, tracer="matter")
linpk_CC = ccl.linear_matter_power(cosmo, k.value, 1)

colors = iter(C)

plt.loglog(k, linpk_SL, c= next(colors))
plt.loglog(k, linpk_CC, c= next(colors), ls="--")

In [ ]:
plt.semilogx(k, (linpk_SL.value/linpk_CC -1) * 100)

Extrapolation

In [ ]:
ktest = np.geomspace(1e-5, 50e2)
ptest = np.exp(linear_interpolate(np.log(k.value), np.log(linpk_SL.value), np.log(ktest)))
plt.loglog(k, linpk_SL)
plt.scatter(ktest, ptest, c="red", marker="+", label="SSLimPy")
plt.scatter(ktest, ccl.linear_matter_power(cosmo, ktest, 1), c="blue", marker="+", label="pyCCL")
plt.xlabel(r"$k\,[\mathrm{Mpc}^{-1}]$")
plt.ylabel(r"$P\,[\mathrm{Mpc}^3]$")
plt.legend()

# Star Terms

SSLimPy

In [ ]:
n_Num = 50
xi, wi = roots_legendre(n_Num)

In [ ]:
X_SL = np.empty((*k.shape, *k.shape, n_Num))

for ik1, k1i in enumerate(k):
    for ik2, k2i in enumerate(k):
        for imu, mui in enumerate(xi):

            # Compute over all permutations of the 1113 diagrams
            X_SL[ik1, ik2, imu] = higher_order.vF3(k1i.value, 1.0, 0.0, k1i.value, -1.0, np.pi, k2i.value, xi[imu], 0.0)
X_SL = 12 * np.sum(X_SL * wi, axis=-1)

In [ ]:
T_3111_SL = X_SL * linpk_SL[:, None]**2 * linpk_SL[None, :]
T_3111_SL += T_3111_SL.T

In [ ]:
T_3111_SL = T_3111_SL

Pyccl

In [ ]:
def funcT_3111_CC(cosmo: ccl.Cosmology):
    k_use = k.value
    pk2d = cosmo.get_linear_power()

    kk = k_use[None, :]
    kp = k_use[:, None]

    def get_X():
        k = kk
        r = kp / k

        def integ(theta):
            cth = np.cos(theta)
            kr2 = k ** 2 + kp ** 2 + 2 * k * kp * cth
            kr = np.sqrt(kr2)
            intd = (5 * r + (7 - 2*r**2)*cth) / (1 + r**2 + 2*r*cth) * \
                   (3/7. * r + 0.5 * (1 + r**2) * cth + 4/7. * r * cth**2)
            # When kr = 0, r = 1 and intd = 0
            intd[np.where(kr == 0)] = 0
            return intd

        isotropized_integ = \
            quad_vec(integ, 0, np.pi)[0] / np.pi

        X = -7./4. * (1 + r**2) + isotropized_integ

        return X

    X = get_X()
    pk = pk2d(k_use,1)[None, :]

    t1113 = 4/9. * pk**2 * pk.T * X
    t1113 += t1113.T
    return t1113, X

In [ ]:
T_3111_CC, X = funcT_3111_CC(cosmo)
X = X * 4/ 9

In [ ]:
n_Num = 200
xi, wi = roots_legendre(n_Num)

In [ ]:
Pkr = np.empty((*k.shape, *k.shape, n_Num))
F2 = np.empty((*k.shape, *k.shape, n_Num))

for ik1, k1i in enumerate(k.value):
    for ik2, k2i in enumerate(k.value):
        for imu, mui in enumerate(xi):
            kr, mur, phr = addVectors(k1i, 1.0, 0.0, k2i, xi[imu], 0.0)
            if not np.isclose(kr, 0.0):
                Pkr[ik1, ik2, imu] = np.exp(
                    linear_interpolate(
                        np.log(k.value),
                        np.log(linpk_SL.value),
                        np.log([kr])
                    )[0]
                )
                F2[ik1, ik2, imu] = higher_order.vF2(k1i, 1, 0.0, kr, -mur, phr+np.pi)

            else:
                Pkr[ik1, ik2, imu] = 0
                F2[ik1, ik2, imu] = 13./28.

P4A_SL_th = Pkr * F2**2
P4X_SL_th = Pkr * F2 * np.transpose(F2, (1,0,2))

P4A_SL = np.sum(wi * P4A_SL_th, axis = -1)
P4X_SL = np.sum(wi * P4X_SL_th, axis = -1)

In [ ]:
T_2211_A_SL = 8 * linpk_SL[:, None]**2 * P4A_SL
T_2211_A_SL += T_2211_A_SL.T

T_2211_X_SL = 8 * linpk_SL[:, None]* linpk_SL[None, :] * P4X_SL
T_2211_X_SL += T_2211_X_SL.T

In [ ]:
from numba import njit

In [ ]:
def collapsed_Trispectrum_LO(
    Lmb1, Lmb2, LmbG2, Lmb3, LmbdG2, LmbG3, LmbDG2, f,
    k1, mu1, ph1,
    k2, mu2, ph2,
    kgrid, Pgrid,
    ):
    k12, mu12, ph12 = addVectors(k1, mu1, ph1, k2, mu2, ph2)
    
    logvk = np.log(np.array([k1, k2, k12]))
    logvPk = linear_interpolate(np.log(kgrid), np.log(Pgrid), logvk)
    vPk = np.exp(logvPk)

    A = 0
    X = 0
    if not np.isclose(k12, 0, atol=1e-12):
        A = (
            8 * vPk[0]**2 * higher_order.vZ1(Lmb1, f, k1, mu1, ph1)**2
            * vPk[2] * higher_order.vZ2(Lmb1, Lmb2, LmbG2, f, k1, -mu1, ph1 + np.pi, k12, mu12, ph12)**2
            + 8 * vPk[1]**2 * higher_order.vZ1(Lmb1, f, k2, mu2, ph2)**2
            * vPk[2] * higher_order.vZ2(Lmb1, Lmb2, LmbG2, f, k2, -mu2, ph2 + np.pi, k12, mu12, ph12)**2
        )
        X = (
            16 * vPk[0] * higher_order.vZ1(Lmb1, f, k1, mu1, ph1)
            * vPk[1] * higher_order.vZ1(Lmb1, f, k2, mu2, ph2)
            * vPk[2] * higher_order.vZ2(Lmb1, Lmb2, LmbG2, f, k1, -mu1, ph1 + np.pi, k12, mu12, ph12)
            * higher_order.vZ2(Lmb1, Lmb2, LmbG2, f, k2, -mu2, ph2 + np.pi, k12, mu12, ph12)
        )

    Star = (
        12 * higher_order.vZ1(Lmb1, f, k1, mu1, ph1)**2 * vPk[0]**2
        * higher_order.vZ1(Lmb1, f, k2, mu2, ph2) * vPk[1]
        * higher_order.vZ3(Lmb1, Lmb2, LmbG2, Lmb3, LmbdG2, LmbG3, LmbDG2, f, k1, mu1, ph1, k1, -mu1, ph1 + np.pi, k2, mu2, ph2)
        + 12 * higher_order.vZ1(Lmb1, f, k2, mu2, ph2)**2 * vPk[1]**2
        * higher_order.vZ1(Lmb1, f, k1, mu1, ph1) * vPk[0]
        * higher_order.vZ3(Lmb1, Lmb2, LmbG2, Lmb3, LmbdG2, LmbG3, LmbDG2, f, k2, mu2, ph2, k1, -mu2, ph2 + np.pi, k1, mu1, ph1)
    )
    return (
        A
        + X
        + Star
    )

def X_SL_int(
    Lmb1, Lmb2, LmbG2, f,
    k1, mu1, ph1,
    k2, mu2, ph2,
    kgrid, Pgrid,
    ):

    k12, mu12, ph12 = addVectors(k1, mu1, ph1, k2, mu2, ph2)
    
    logvk = np.log(np.array([k1, k2, k12]))
    logvPk = linear_interpolate(np.log(kgrid), np.log(Pgrid), logvk)
    vPk = np.exp(logvPk)

    X = 0
    if not np.isclose(k12, 0, atol=1e-12):
        X = (
            vPk[2] # 16 * vPk[0] * vPk[1]
            * higher_order.vZ1(Lmb1, f, k1, mu1, ph1)
            * higher_order.vZ1(Lmb1, f, k2, mu2, ph2)
            * higher_order.vZ2(Lmb1, Lmb2, LmbG2, f, k1, -mu1, ph1 + np.pi, k12, mu12, ph12)
            * higher_order.vZ2(Lmb1, Lmb2, LmbG2, f, k2, -mu2, ph2 + np.pi, k12, mu12, ph12)
        )
    return X


# Snake - Terms


Pyccl

In [ ]:
k_use = k.value
pk2d = cosmo.get_linear_power()

kk = k_use[None, :]
kp = k_use[:, None]
nk = len(k_use)

# Get P4A, P4X
def integ(theta):
    cth = np.cos(theta)
    kr2 = kk ** 2 + kp ** 2 + 2 * kk * kp * cth
    kr = np.sqrt(kr2)

    f2 = 5./7. - 0.5 * (1 + kk ** 2 / kr2) * (1 + kp / kk * cth) + \
        2/7. * kk ** 2 / kr2 * (1 + kp / kk * cth)**2
    f2[np.where(kr == 0)] = 13. / 28

    pkr = pk2d(kr.flatten(), 1.0, cosmo).reshape((nk, nk))
    
    return np.array([pkr * f2**2, pkr * f2 * f2.T])

P4A_CL, P4X_CL = quad_vec(integ, 0, np.pi)[0] / np.pi

X_int_CC = []
for imu, mui in enumerate(xi):
    A, X = integ(np.arccos(mui))
    X_int_CC.append(X)
X_int_CC = np.array(X_int_CC)

In [ ]:
X_test_CC = np.sum(X_int_CC[:,15,:] * wi[:, None], axis=0)

In [ ]:
X_int_SL = np.empty((*k.shape, n_Num))
for ik1, k1i in enumerate(k):
    for imu, mui in enumerate(xi):
        X_int_SL[ik1, imu] = X_SL_int(1, 0, 0, 0, k1i.value, 1.0, 0.0, k.value[15], mui, 0.0, k.value, linpk_SL.value)
X_SL = np.sum(X_int_SL * wi, axis=-1)



In [ ]:
colors = iter(C)

fig, axs = plt.subplots(2, 1, sharex=True)

indicies = [10,15,30]
for i in indicies:
    c = next(colors)
    axs[0].semilogy(xi, X_int_SL[i,:], c=c)
    axs[1].semilogy(xi, -X_int_SL[i,:], c=c)

    c = next(colors)
    axs[0].semilogy(xi, X_int_CC[:,15, i], c=c, ls="--")
    axs[1].semilogy(xi, -X_int_CC[:,15, i], c=c, ls="--")

axs[1].invert_yaxis()
plt.plot([],[], "k--", label="pyccl")
plt.plot([],[], "k-", label="SSLimPy")

plt.legend()

In [ ]:
plt.loglog(k, X_test_CC)

In [ ]:
P4X_CL_th = np.empty((*k.shape, *k.shape, n_Num))
P4A_CL_th = np.empty((*k.shape, *k.shape, n_Num))

for ith, mui in enumerate(xi):
    thi = np.arccos(mui)
    A, X = integ(thi)
    P4A_CL_th[:,:, ith] = A
    P4X_CL_th[:,:, ith] = X

In [ ]:
T_2211_A_CL = 8 * linpk_CC[None, :]**2 * P4A_CL
T_2211_A_CL += T_2211_A_CL.T

T_2211_X_CL = 8 * linpk_CC[:, None]* linpk_CC[None, :] * P4X_CL
T_2211_X_CL += T_2211_X_CL.T

In [ ]:
colors = iter(C)

plt.loglog(k, T_2211_A_SL[:,15], c=next(colors))
plt.loglog(k, T_2211_A_CL[:,15], c=next(colors), ls="--")
plt.loglog(k, T_2211_A_SL[:,45], c=next(colors))
plt.loglog(k, T_2211_A_CL[:,45], c=next(colors), ls="--")
plt.loglog(k, T_2211_A_SL[:,75], c=next(colors))
plt.loglog(k, T_2211_A_CL[:,75], c=next(colors), ls="--")

plt.loglog([],[], "k--", label="pyccl")
plt.loglog([],[], "k-", label="SSLimPy")
plt.legend()

In [ ]:
colors = iter(C)

fig, axs = plt.subplots(2, 1, sharex=True)

indicies = [15, 45, 75]
for i in indicies:
    c = next(colors)
    axs[0].loglog(k, T_2211_X_SL[:,i], c=c)
    axs[1].loglog(k, -T_2211_X_SL[:,i], c=c)

    c = next(colors)
    axs[0].loglog(k, T_2211_X_CL[:,i], c=c, ls="--")
    axs[1].loglog(k, -T_2211_X_CL[:,i], c=c, ls="--")

axs[1].invert_yaxis()
plt.loglog([],[], "k--", label="pyccl")
plt.loglog([],[], "k-", label="SSLimPy")

plt.legend()

In [ ]:
for i in range(98):
    if T_2211_X_SL[i,15]>0 and T_2211_X_SL[i+1,15]<0:
        print(i)

In [ ]:
k[36]

In [ ]:
T_2211_SL = T_2211_A_SL + T_2211_X_SL
T_2211_CC = T_2211_A_CL + T_2211_X_CL

In [ ]:
colors = iter(C)

plt.loglog(k, T_2211_SL[:,15], c=next(colors), ls="-")
plt.loglog(k, T_2211_CC[:,15], c=next(colors), ls="--")

plt.loglog(k, T_2211_SL[:,45], c=next(colors), ls="-")
plt.loglog(k, T_2211_CC[:,45], c=next(colors), ls="--")

plt.loglog(k, T_2211_SL[:,75], c=next(colors), ls="-")
plt.loglog(k, T_2211_CC[:,75], c=next(colors), ls="--")

plt.xlabel(r"$k\,[\mathrm{Mpc}^{-1}]$")
plt.ylabel(r"$\langle T_{1122} \rangle\,[\mathrm{Mpc}^6]$")

plt.loglog([],[], "k--", label="pyccl")
plt.loglog([],[], "k-", label="SSLimPy")
plt.legend()


In [ ]:
colors = iter(C)

plt.loglog(k, -T_3111_SL[:,15], c=next(colors), ls="-")
plt.loglog(k, -T_3111_CC[:,15], c=next(colors), ls="--")

plt.loglog(k, -T_3111_SL[:,45], c=next(colors), ls="-")
plt.loglog(k, -T_3111_CC[:,45], c=next(colors), ls="--")

plt.loglog(k, -T_3111_SL[:,75], c=next(colors), ls="-")
plt.loglog(k, -T_3111_CC[:,75], c=next(colors), ls="--")

plt.xlabel(r"$k\,[\mathrm{Mpc}^{-1}]$")
plt.ylabel(r"$\langle T_{1113} \rangle\,[\mathrm{Mpc}^6]$")

plt.loglog([],[], "k--", label="pyccl")
plt.loglog([],[], "k-", label="SSLimPy")
plt.legend()


# Full expression

In [ ]:
# F = np.loadtxt("CCL_Trispectrum.txt")
T_CC = T_2211_X_CL + T_2211_A_CL + T_3111_CC
T_SL = T_2211_X_SL.value + T_2211_A_SL.value + T_3111_SL.value

In [ ]:
colors = iter(C)

plt.loglog(k, T_SL[:, 25], c=next(colors))
plt.loglog(k, T_CC[:, 25], c=next(colors), ls="--")

plt.loglog(k, T_SL[:, 45], c=next(colors))
plt.loglog(k, T_CC[:, 45], c=next(colors), ls="--")

plt.loglog(k, T_SL[:, 75], c=next(colors))
plt.loglog(k, T_CC[:, 75], c=next(colors), ls="--")

plt.xlabel(r"$k\,[\mathrm{Mpc}^{-1}]$")
plt.ylabel(r"$\langle T \rangle\,[\mathrm{Mpc}^6]$")

plt.loglog([],[], "k--", label="pyccl")
plt.loglog([],[], "k-", label="SSLimPy")
plt.legend()

In [ ]:
args = dict(b1=1, b2=0, b3=0, bG2=0, bG3=0, bdG2=0, bDG2=0, f=0, sigma_parr=0, sigma_perp=0)
a = cov.integrate_4h(args=args, eps=1e-2, z=0.0)

In [ ]:
colors = iter(C)
indicies = [15, 45, 75]
for i in indicies:
    c = next(colors)
    plt.loglog(k, a[:, i, 0, 0],c=c)
    c = next(colors)
    plt.loglog(k, -a[:, i, 0, 0],ls="--",c=c)

In [ ]:
T_CC = np.loadtxt("/home/sefa/Desktop/LIM-Code/SSLimPy/CCL_Trispectrum.txt")

In [ ]:
colors = iter(C)

plt.loglog(k, a[:, 25, 0, 0], c=next(colors))
plt.loglog(k, T_CC[:, 25], c=next(colors), ls="--")

plt.loglog(k, a[:, 45, 0, 0], c=next(colors))
plt.loglog(k, T_CC[:, 45], c=next(colors), ls="--")

plt.loglog(k, a[:, 75, 0, 0], c=next(colors))
plt.loglog(k, T_CC[:, 75], c=next(colors), ls="--")

plt.xlabel(r"$k\,[\mathrm{Mpc}^{-1}]$")
plt.ylabel(r"$\langle T \rangle\,[\mathrm{Mpc}^6]$")

plt.loglog([],[], "k--", label="pyccl")
plt.loglog([],[], "k-", label="SSLimPy")
plt.legend()